In [16]:
import pandas as pd
import numpy as np
from joblib import dump
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import normalize, LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC

df = pd.read_csv('dermatology.csv')
df = df.replace('?', np.nan)
df.dropna(inplace=True)

Y = df['class']
X = df.drop('class', axis = 1)

pca = PCA(n_components = 'mle') #decide o melhor número de forma automatizada
pca.fit(X)
X = pca.transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15,
random_state=42)
# print(Y_test)

# model = GridSearchCV(
#     tree.DecisionTreeClassifier(random_state=100),
#     {
#         'max_depth' : [4, 5, 6],
#         'min_samples_split' : [2, 3],
#         'min_samples_leaf': [4, 5, 6]
#     },
#     n_jobs = 4
# )

model = GridSearchCV(
    SVC(),
    {
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree' : list(range(2, 5))
    },
    n_jobs = 5
)

model.fit(X_train, Y_train)


print(accuracy_score(Y_test, model.predict(X_test)))
print(accuracy_score(Y_train, model.predict(X_train)))

0.9629629629629629
0.993421052631579


In [4]:
print(model.best_params_)

{'max_depth': 6, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 2}
